In [28]:
# import libraries
import numpy as np
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from skimage import io,color
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:
# load images
path = 'surprise'
surprise = []
for file in os.listdir(path):
    img = io.imread(path + "\\"+ file, as_gray=True)
    rows,cols = img.shape
    img_size = rows * cols
    img_1d = img.reshape(img_size)
    surprise.append(img_1d)

surprise = np.array(surprise)

path = 'happy'
happy = []
for file in os.listdir(path):
    img = io.imread(path + "\\"+ file, as_gray=True)
    rows,cols = img.shape
    img_size = rows * cols
    img_1d = img.reshape(img_size)
    happy.append(img_1d)
happy = np.array(happy)

path = 'disgust'
disgust = []
for file in os.listdir(path):
    img = io.imread(path + "\\"+ file, as_gray=True)
    rows,cols = img.shape
    img_size = rows * cols
    img_1d = img.reshape(img_size)
    disgust.append(img_1d)
disgust = np.array(disgust)

    
path = 'neutral'
neutral = []
for file in os.listdir(path):
    img = io.imread(path + "\\"+ file, as_gray=True)
    rows,cols = img.shape
    img_size = rows * cols
    img_1d = img.reshape(img_size)
    neutral.append(img_1d)    
neutral = np.array(neutral)
print('Shape of dataset:')
print(neutral.shape)
print(surprise.shape)
print(happy.shape)
print(disgust.shape)

Shape of dataset:
(190, 180000)
(88, 180000)
(85, 180000)
(47, 180000)


In [7]:
# split data into train and test subset
neutral_train, neutral_test = train_test_split(neutral, train_size =120, random_state=42)
surprise_train, surprise_test = train_test_split(surprise, train_size =60, random_state=42)
happy_train, happy_test = train_test_split(happy, train_size =60, random_state=42)
disgust_train, disgust_test = train_test_split(disgust, train_size =30, random_state=42)

In [8]:
# create data arrays
X_train = np.append(neutral_train, surprise_train, axis=0)
X_train = np.append(X_train, happy_train, axis=0)
X_train = np.append(X_train, disgust_train, axis=0)

X_test = np.append(neutral_test, surprise_test, axis=0)
X_test = np.append(X_test, happy_test, axis=0)
X_test = np.append(X_test, disgust_test, axis=0)

In [10]:
# create labels array
y_train = np.append(np.zeros((120,1)), np.ones((60,1)), axis=0)
y_train = np.append(y_train, 2 * np.ones((60,1)), axis=0)
y_train = np.append(y_train, 3 * np.ones((30,1)), axis=0)

y_test = np.append(
    np.zeros((neutral_test.shape[0], 1)),
    np.ones((surprise_test.shape[0], 1)),axis=0)
y_test = np.append(
    y_test, 2 * np.ones((happy_test.shape[0], 1)),axis=0)
y_test = np.append(
    y_test, 3 * np.ones((disgust_test.shape[0], 1)),axis=0)    
    
print('Shape of x-train:',X_train.shape)
print('Shape of y-train:',y_train.shape)
print('Shape of x-train:',X_test.shape)
print('Shape of y-test:',y_test.shape)

Shape of x-train: (270, 180000)
Shape of y-train: (270, 1)
Shape of x-train: (140, 180000)
Shape of y-test: (140, 1)


In [14]:
# apply PCA
pca = PCA(n_components=40)
pca.fit(X_train)
X_train_reduced = pca.transform(X_train)
X_test_reduced = pca.transform(X_test)
print('Shape of Reduced Data:')
print(X_train_reduced.shape)
print(X_test_reduced.shape)

Shape of Reduced Data:
(270, 40)
(140, 40)


In [17]:
# surprise vs other
clf_surprise = svm.SVC()
loc  = np.where(y_train!=1)
y = y_train.copy()
y[loc] = 0*y[loc]

loc = np.where(y_test!=1)
y_test1 = y_test.copy()
y_test1[loc] = 0*y_test1[loc]

clf_surprise.fit(X_train_reduced, y.ravel())
y_test_pred = clf_surprise.predict(X_test_reduced)

mse_surprised=mean_squared_error(y_test_pred, y_test1)
acc_surprised = accuracy_score(y_test_pred, y_test1)

print('MSE_surprised:', mse_surprised)
print('acc_surprised:', acc_surprised)

MSE_surprised: 0.10714285714285714
acc_surprised: 0.8928571428571429


In [19]:
# happy vs other
from sklearn import svm
clf_happy = svm.SVC()
loc  = np.where(y_train != 2)
y = y_train.copy()
y[loc ] = 0 *y[loc]

loc  = np.where(y_test != 2)
y_test1 = y_test.copy()
y_test1[loc ] = 0 *y_test1[loc]

clf_happy.fit(X_train_reduced, y.ravel())
y_test_pred=clf_happy.predict(X_test_reduced)

mse_happy = mean_squared_error(y_test_pred//2,y_test1//2)
acc_happy = accuracy_score(y_test_pred, y_test1)
print('MSE_happy:',mse_happy)
print('acc_happy:',acc_happy)

MSE_happy: 0.15714285714285714
acc_happy: 0.8428571428571429


In [23]:
# disgust vs other
clf_disgust = svm.SVC()
loc = np.where(y_train!=3)
y = y_train.copy()
y[loc ] = 0*y[loc]

loc = np.where(y_test != 3)
y_test1 = y_test.copy()
y_test1[loc] = 0*y_test1[loc]

clf_disgust.fit(X_train_reduced, y.ravel())
y_test_pred=clf_disgust.predict(X_test_reduced)

mse_disgust = mean_squared_error(y_test_pred//3, y_test1//3)
acc_disgust = accuracy_score(y_test_pred, y_test1)
print('MSE_disgust:', mse_disgust)
print('acc_disgust:', acc_disgust)

MSE_disgust: 0.12142857142857143
acc_disgust: 0.8785714285714286


In [25]:
clf = svm.SVC()

clf.fit(X_train_reduced, y_train.ravel())
y_test_pred=clf.predict(X_test_reduced)

mse=mean_squared_error(y_test_pred,y_test)
acc = accuracy_score(y_test_pred, y_test)

print('MSE:', mse)
print('acc:', acc)

MSE: 1.85
acc: 0.5857142857142857


In [27]:
# create confusion matrix
confusion_matrix(y_test, y_test_pred)

array([[60,  0, 10,  0],
       [14, 14,  0,  0],
       [17,  0,  8,  0],
       [15,  0,  2,  0]], dtype=int64)

In [32]:
precision = precision_score(y_test, y_test_pred, average='weighted')
print('Precision Score:', precision)
recall = recall_score(y_test, y_test_pred, average='weighted')
print('Recall Score:', recall)

Precision Score: 0.5544474393530997
Recall Score: 0.5857142857142857


C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
data = (np.append(X_train_reduced , y_train , axis = 1))
max_acc = 0
c_max = 0
for c in [0.0001,0.001,0.01,0.1]: 
    mse=0
    acc=0
    for k in range(0,270,30):
        data_test = data[k:k+30]
        data_train = np.delete(data, np.arange(k,k+30) , axis=0)
        X_train_reduced_C=data_train[:,0:40]
        y_train_C=data_train[:,40]
        X_test_reduced_C=data_test[:,0:40]
        y_test_C=data_test[:,40]
        clf_step2 = svm.SVC(C = c,kernel='linear')
        clf_step2.fit(X_train_reduced_C, y_train_C)
        y_test_pred_C=clf_step2.predict(X_test_reduced_C)
        MSE=mean_squared_error(y_test_pred_C,y_test_C)
        mse=mse+MSE
        Acc = accuracy_score(y_test_pred_C, y_test_C)
        acc= acc + Acc
        
    print('MSE=',mse/9)
    print('acc=',acc/9)
    if acc/9>max_acc:
        c_max = c
        max_acc = acc/9
print(max_acc , c_max)

MSE= 1.462962962962963
acc= 0.6444444444444446
MSE= 1.462962962962963
acc= 0.6444444444444446
MSE= 1.462962962962963
acc= 0.6444444444444446
MSE= 1.462962962962963
acc= 0.6444444444444446
0.6444444444444446 0.0001


In [34]:
clf = svm.LinearSVC(C = c_max)
clf.fit(X_train_reduced , y_train)
y_test_pred=clf.predict(X_test_reduced)
mse=mean_squared_error(y_test_pred,y_test)

Acc = accuracy_score(y_test_pred, y_test)
print('MSE:', mse)
print('acc:', Acc)

MSE: 1.4785714285714286
acc: 0.6928571428571428


C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = [
  
  {'C': [0.0001, 0.001 , 0.01 , 0.1 , 1 , 10 , 100, 1000], 'gamma': [1e-12, 1e-10, 1e-8, 1e-6, 1e-4, 1e-2 ,1,10], 'kernel': ['rbf']},
 ]
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train_reduced, y_train.ravel())

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END .C=0.0001, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END .C=0.0001, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END .C=0.0001, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END .C=0.0001, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END .C=0.0001, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END .C=0.0001, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END .C=0.0001, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END .C=0.0001, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END .C=0.0001, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END .C=0.0001, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END .C=0.0001, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END .C=0.0001, gamma=1e-08, kernel=rbf

[CV 3/5] END ....C=0.1, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END ....C=0.1, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END ....C=0.1, gamma=1e-10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END ....C=0.1, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END ....C=0.1, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END ....C=0.1, gamma=1e-08, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1e-06, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1e-06, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END ....C=0.1, gamma=1e-06, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END ....C=0.1, gamma=1e-06, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END ....C=0.1, gamm

[CV 3/5] END .........C=10, gamma=1, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END .........C=10, gamma=1, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END .........C=10, gamma=1, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END ........C=10, gamma=10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END ........C=10, gamma=10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END ........C=10, gamma=10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END ........C=10, gamma=10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END ........C=10, gamma=10, kernel=rbf;, score=0.444 total time=   0.0s
[CV 1/5] END ....C=100, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 2/5] END ....C=100, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 3/5] END ....C=100, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 4/5] END ....C=100, gamma=1e-12, kernel=rbf;, score=0.444 total time=   0.0s
[CV 5/5] END ....C=100, gamm

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'gamma': [1e-12, 1e-10, 1e-08, 1e-06, 0.0001, 0.01, 1,
                                    10],
                          'kernel': ['rbf']}],
             verbose=3)

In [36]:
print(grid.best_params_)
clf = grid.best_estimator_
y_pred = clf.predict(X_test_reduced)
mse=mean_squared_error(y_pred,y_test)

Acc = accuracy_score(y_pred, y_test)
print('MSE:', mse)
print('acc:', Acc)

{'C': 100, 'gamma': 1e-10, 'kernel': 'rbf'}
MSE: 0.7285714285714285
acc: 0.8571428571428571
